In [2]:
"""Entry point to evolving the neural network. Start here."""
import logging
from optimizer import Optimizer
from tqdm import tqdm

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt'
)

ModuleNotFoundError: No module named 'optimizer'

In [ ]:
def train_networks(networks):
    
    """Train each network.
    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """
    pbar = tqdm(total=len(networks))
    
    for network in networks:
        network.train()
        pbar.update(1)
        
    pbar.close()

In [ ]:
def get_average_accuracy(networks):
    """Get the average accuracy for a group of networks.
    Args:
        networks (list): List of networks
    Returns:
        float: The average accuracy of a population of networks.
    """
    total_accuracy = 0
    for network in networks:
        total_accuracy += network.accuracy

    return total_accuracy / len(networks)

In [ ]:
def generate(generations, population, nn_param_choices):
    """Generate a network with the genetic algorithm.
    Args:
        generations (int): Number of times to evole the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating
    """
    optimizer = Optimizer(nn_param_choices)
    networks = optimizer.create_population(population)

    # Evolve the generation.
    for i in range(generations):
        logging.info("***Doing generation %d of %d***" %
                     (i + 1, generations))

        # Train and get accuracy for networks.
        train_networks(networks)

        # Get the average accuracy for this generation.
#         average_accuracy = get_average_accuracy(networks)
        average_accuracy = optimizer.grade_overall(networks)
        top_five_average_accuracy = optimizer.grade_top_five(networks)
        
        # Print out the average accuracy each generation.
        logging.info("Generation overall average: %.2f%%" % (average_accuracy * 100))
        logging.info("Generation top five average: %.2f%%" % (top_five_average_accuracy * 100))
        logging.info('-'*80)

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)

    # Sort our final population.
    networks = sorted(networks, key=lambda x: x.accuracy, reverse=True)

    # Print out the top 5 networks.
    print_networks(networks[:5])
    
    return networks

In [ ]:
def print_networks(networks):
    """Print a list of networks.
    Args:
        networks (list): The population of networks
    """
    logging.info('-'*80)
    for network in networks:
        network.print_network()

In [ ]:
def main():
    """Evolve a network."""
    generations = 20  # Number of times to evole the population.
    population = 20  # Number of networks in each generation.
#     dataset = 'cifar10'

    nn_param_choices = {
        'window_size':[i for i in range(1,50)],
        'nb_neurons': [i for i in range(3, 41, 1)],
        'nb_layers': [i for i in range(1,11)],
        'batch_size':[i for i in range(1,21)],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam','ftrl']
    }

    logging.info("***Evolving %d generations with population %d***" %
                 (generations, population))

    networks = generate(generations, population, nn_param_choices)

    return networks

In [ ]:
if __name__ == '__main__':
    networks = main()

In [ ]:
for network in networks:
    print(network.accuracy)